In [27]:
import pandas as pd
import numpy as np
import math

## THE FOLLOWING IS USING DATA FROM THE WEBSITE - FOR NOW WE'LL JUST USE THE DATA LOCALLY
But we'll keep this section here for now, commented out.

In [2]:
# bigoldlist = ['2013-02.json',
# '2013-03.json',
# '2013-04.json',
# '2013-05.json',
# '2013-06.json',
# '2013-07.json',
# '2013-08.json',
# '2013-09.json',
# '2013-10.json',
# '2013-11.json',
# '2013-12.json',
# '2014-01.json',
# '2014-02.json',
# '2014-03.json',
# '2014-04.json',
# '2014-05.json',
# '2014-06.json',
# '2014-07.json',
# '2014-08.json',
# '2014-09.json',
# '2014-10.json',
# '2014-11.json',
# '2014-12.json',
# '2015-01.json',
# '2015-02.json',
# '2015-03.json',
# '2015-04.json',
# '2015-05.json',
# '2015-06.json',
# '2015-07.json',
# '2015-08.json',
# '2015-09.json',
# '2015-10.json',
# '2015-11.json',
# '2015-12.json',
# '2016-01.json',
# '2016-02.json',
# '2016-03.json',
# '2016-04.json',
# '2016-05.json',
# '2016-06.json',
# '2016-07.json',
# '2016-08.json',
# '2016-09.json',
# '2016-10.json',
# '2016-11.json',
# '2016-12.json',
# '2017-01.json',
# '2017-02.json',
# '2017-03.json',
# '2017-04.json',
# '2017-05.json',
# '2017-06.json',
# '2017-07.json',
# '2017-08.json',
# '2017-09.json',
# '2017-10.json',
# '2017-11.json',
# '2017-12.json',
# '2018-01.json',
# '2018-02.json',
# '2018-03.json',
# '2018-04.json',
# '2018-05.json',
# '2018-06.json',
# '2018-07.json',
# '2018-08.json',
# '2018-09.json',
# '2018-10.json',
# '2018-11.json',
# '2018-12.json',
# '2019-01.json',
# '2019-02.json',
# '2019-03.json',
# '2019-04.json',
# '2019-05.json',
# '2019-06.json',
# '2019-07.json',
# '2019-08.json',
# '2019-09.json',
# '2019-10.json',
# '2019-11.json',
# '2019-12.json',
# '2020-01.json',
# '2020-02.json',
# '2020-03.json',
# '2020-04.json',
# '2020-05.json',
# '2020-06.json',
# '2020-07.json',
# '2020-08.json',
# '2020-09.json',
# '2020-10.json',
# '2020-11.json',
# '2020-12.json',
# '2021-01.json',
# '2021-02.json',
# '2021-03.json',
# '2021-04.json']

In [3]:
# # Test start of web pipeline (not used)
# url = 'https://terra.snellman.net/data/events/' + bigoldlist[0]
# eventsdata1 = pd.read_json(url)
# print(len(eventsdata1))
# eventsdata1.head()

In [4]:
# # Load in the data remotely
# for gameset in bigoldlist:
#     url = 'https://terra.snellman.net/data/events/' + gameset
#     eventsdata = pd.read_json(url)
    
#     for eachgame in range(len(gameset)):
#         singlerow = eventsdata.iloc[[eachgame]]
#         newcol = get_vp_from_game(gameEventRow)
#         vpdf = vpdf.append(newdf, ignore_index=True)      

## Loading in the data Locally

In [5]:
folderlocation = "D:/PycharmProjects/TerraBot/terra-mystica"
gameevents = pd.read_csv(f'{folderlocation}/game_events.csv')

## Creating the new dataset

In [18]:
# make dataframe to be added to
def makenewdf():
    validfactions = ['witches', 'auren', 'swarmlings', 'mermaids', 'cultists', 'halflings', 'dwarves', 'engineers', 'chaosmagicians', 'giants', 'fakirs', 'nomads', 'darklings', 'alchemists']
    dfcols = ['game'] + validfactions
    vpdf = pd.DataFrame(columns=dfcols)
    
    return vpdf

vpdf = makenewdf()

sensecheck = False
if sensecheck:
    newdf = pd.DataFrame([[np.nan] * 15], columns=dfcols)
    newdf['auren'].replace({np.nan: 'test'}, inplace=True)
    vpdf = vpdf.append(newdf, ignore_index=True)
    print(len(vpdf))
    vpdf.head()

In [46]:
def get_vp_from_game(singleGameEvents):
    """Input game events for a single game. This is a pd.DataFrame. 
    Output a row where each faction in the game has its vp populated (the rest are nans)
    """
    newdf = pd.DataFrame([[np.nan] * 15], columns=dfcols)
    
    # assign the game number
    gameno = list(pd.unique(singleGameEvents['game']))
    # assert len(gameno) == 1, 'More than 1 unique game was found'
    
    
    try:
        newdf['game'].replace({np.nan: gameno[0]}, inplace=True)
    except:
        print(f'DEBUGGING: len of table is {len(singleGameEvents)}')
        print(f'DEBUGGING: gamnos list: {gameno}')
        print(singleGameEvents)
        raise
        
    # find factions
    verifiedfactions = []
    rawfactions = list(pd.unique(singleGameEvents['faction']))
    
    # there are some artifacts in the data. We need to filter them out.
    for rawfaction in rawfactions:
        if rawfaction in validfactions:
            verifiedfactions.append(rawfaction)
    
    for faction in verifiedfactions:
        vpfaction = sum(singleGameEvents[(singleGameEvents['event'] == 'vp') & (singleGameEvents['faction'] == faction)]['num'])
        newdf[faction].replace({np.nan: vpfaction}, inplace=True)
        
    return newdf

In [10]:
# get some stats
gameslist = list(pd.unique(gameevents['game']))
print(f'Number of game events: {len(gameevents)}')
print(f"Number of games: {len(gameslist)}")

Number of game events: 73419074
Number of games: 115612


In [9]:
# sense check
import time

start = time.time()
singlegame = gameevents[gameevents['game'] == '0000000001']
singlegamefactions = (list(pd.unique(singlegame['faction'])))
vpforgame = get_vp_from_game(singlegame)
end = time.time()

print(f'Time elapsed: {end-start}s')
print(f'Time to do all games at current speed: {((end-start)*len(pd.unique(gameevents["game"]))/3600)}hrs')
vpforgame

Time elapsed: 3.5411174297332764s
Time to do all games at current speed: 113.72101896842321hrs


,game,witches,auren,swarmlings,mermaids,cultists,halflings,dwarves,engineers,chaosmagicians,giants,fakirs,nomads,darklings,alchemists
0,0000000001,NaN,NaN,NaN,118.0,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# that's way too long. How long will it take to split the table into 1500, 100 game tables and work on those?
first100games = gameslist[:100]

start = time.time()
smalltbl = gameevents[gameevents['game'].isin(first100games)]
end = time.time()
smalltabletime = end-start

print(f'Time elapsed to make small table: {smalltabletime}s')

start = time.time()
singlegame = smalltbl[smalltbl['game'] == '0000000001']
singlegamefactions = (list(pd.unique(singlegame['faction'])))
vpforgame = get_vp_from_game(singlegame)
end = time.time()
datacreatetime = end-start

print(f'Time elapsed to get data for single game now: {end-start}s')
totalsmalltabletime = smalltabletime * len(pd.unique(gameevents["game"]))/(100 * 3600)  # divide by 100 as that's the size of table, divide by 3600 for s -> hrs
totaldatacreatetime = datacreatetime * len(pd.unique(gameevents["game"]))/3600
print(f'Time elapsed to do all small tables: {totalsmalltabletime}hrs')
print(f'Time elapsed to do all data creation: {totaldatacreatetime}hrs')
print(f'Time elapsed to do all processing: {totalsmalltabletime + totaldatacreatetime}hrs')


Time elapsed to make small table: 1.23183012008667s
Time elapsed to get data for single game now: 0.006997346878051758s
Time elapsed to do all small tables: 0.39559539956516687hrs
Time elapsed to do all data creation: 0.22471590757369994hrs
Time elapsed to do all processing: 0.6203113071388668hrs


Damn. I've just saved ~200 times the amount of time by doing that. As a complete aside, for pure interest, I wonder what the best trade-off between spitting into smaller & smaller tables (which increases time to do this) is vs saving time to then filter with smaller tables, is? (will check this out later)

In [49]:
# sense check for 100 games
vpdf = makenewdf()
gamelengthlen = len(gameslist)
gamesroundup = math.ceil(gamelengthlen / 100.0) * 100
jj = 0

for ii in range(100, 201, 100):     
    ii = min(ii, gamelengthlen)    
    print(f'Progressed to {ii}th game')
    
    next100games = gameslist[jj:ii]
    jj = ii  # so that we don't get any repetitions at the very end, where our set will be smaller
    
    smalltbl = gameevents[gameevents['game'].isin(next100games)]
    for game in next100games:
        singlegame = smalltbl[smalltbl['game'] == game]
        
        if not len(singlegame) == 0:        
            vpforgame = get_vp_from_game(singlegame)
            vpdf = vpdf.append(vpforgame, ignore_index=True)
    
print(f"no of unique games in table is: {len(list(pd.unique(vpdf['game'])))}")
vpdf

Progressed to 100th game
Progressed to 200th game
no of unique games in table is: 200


,game,witches,auren,swarmlings,mermaids,cultists,halflings,dwarves,engineers,chaosmagicians,giants,fakirs,nomads,darklings,alchemists
0,0000000001,NaN,NaN,NaN,118.0,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,050615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0,NaN
2,0512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.0,101.0,NaN,NaN,NaN,NaN,NaN
3,051501,NaN,84.0,NaN,NaN,NaN,77.0,NaN,NaN,72.0,NaN,NaN,54.0,78.0,NaN
4,060303,NaN,NaN,NaN,108.0,NaN,NaN,NaN,121.0,NaN,NaN,78.0,NaN,128.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,4pLeague_S7_D6L17_G7,NaN,NaN,48.0,NaN,NaN,NaN,NaN,NaN,73.0,NaN,NaN,130.0,87.0,NaN
196,4pLeague_S7_D6L18_G2,NaN,105.0,NaN,NaN,NaN,NaN,NaN,NaN,108.0,NaN,NaN,128.0,53.0,NaN
197,4pLeague_S7_D6L18_G5,NaN,NaN,97.0,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,110.0,117.0,NaN
198,4pLeague_S7_D6L18_G6,NaN,NaN,78.0,NaN,NaN,103.0,NaN,NaN,137.0,NaN,NaN,NaN,61.0,NaN


Looks good! Let's try it with the real thing

In [51]:
vpdf = makenewdf()
gamelengthlen = len(gameslist)
gamesroundup = math.ceil(gamelengthlen / 100.0) * 100
jj = 0

for ii in range(100, gamesroundup+1, 100):     
    ii = min(ii, gamelengthlen)    
    if (ii % 10000) == 0:  # update every 10000 games
        print(f'Progressed to {ii}th game')
    
    next100games = gameslist[jj:ii]
    jj = ii  # so that we don't get any repetitions at the very end, where our set will be smaller
    
    smalltbl = gameevents[gameevents['game'].isin(next100games)]
    for game in next100games:
        singlegame = smalltbl[smalltbl['game'] == game]
        
        if not len(singlegame) == 0:
            vpforgame = get_vp_from_game(singlegame)
            vpdf = vpdf.append(vpforgame, ignore_index=True)

print(f"no of unique games in table is: {len(list(pd.unique(vpdf['game'])))}")
vpdf

Progressed to 10000th game
Progressed to 20000th game
Progressed to 30000th game
Progressed to 40000th game
Progressed to 50000th game
Progressed to 60000th game
Progressed to 70000th game
Progressed to 80000th game
Progressed to 90000th game
Progressed to 100000th game
Progressed to 110000th game
no of unique games in table is: 115611


,game,witches,auren,swarmlings,mermaids,cultists,halflings,dwarves,engineers,chaosmagicians,giants,fakirs,nomads,darklings,alchemists
0,0000000001,NaN,NaN,NaN,118.0,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,050615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0,NaN
2,0512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.0,101.0,NaN,NaN,NaN,NaN,NaN
3,051501,NaN,84.0,NaN,NaN,NaN,77.0,NaN,NaN,72.0,NaN,NaN,54.0,78.0,NaN
4,060303,NaN,NaN,NaN,108.0,NaN,NaN,NaN,121.0,NaN,NaN,78.0,NaN,128.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115606,Youreacrookcaptianhook,NaN,NaN,126.0,NaN,NaN,105.0,NaN,NaN,NaN,NaN,NaN,37.0,NaN,NaN
115607,YourMomIsAChaosMagician,112.0,NaN,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN
115608,ytuwertqwtr,NaN,NaN,105.0,NaN,NaN,NaN,102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115609,yuertyqert,128.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.0,NaN,NaN,NaN,NaN,NaN


Filtering bad data. We'll need to check... 
1. the game has finished
2. the map used (only use a sub-set of maps, or we could one-hot a list of accepted maps in X (features)
3. additional rules? Look up
4. Number of players. Possible for 2, 3, 4 or 5 player games, we could one-hot the differences as there might be similarities in different players that can contribute to predictions between different player games. Although >5 players should be removed.
5. No extra factions (shapeshifters etc)

In [56]:
vpdf.to_csv('D://PycharmProjects/TerraBot/data/factor-picker-bot/rawvpdata.csv')